In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [43]:
results_dir = "all_experiments"
#sub_files = ["momentumTest", "chatAdamTests/checkpoints", "ScheduleTest_constant", "scheduleTest_all4", "1Test", "05Test", "01Test", "simpleCnnTests", "rdmExp_tests", "adrien", "adri_weights_no_reset", "TestTinyCNN", "TestTinyCnnAdam", 'TestTinyCnnAdam_all_input', "TestTinyCnnSgd_Input_all", "TestTinyCNN_every_layer_adam", "TestTinyCNN_every_layer_sgd"]
sub_files = ["TMP_baby_stuck_noise", '1Test', '01Test']
print(len(sub_files))
json_dir = [os.path.join(results_dir, sub_file, "metrics") for sub_file in sub_files]
json_dir += [os.path.join(results_dir, sub_file, "checkpoints/metrics") for sub_file in sub_files]

3


In [44]:
json_files = []
for dir_name in json_dir:
    if not os.path.exists(dir_name):
        print(f"Directory {dir_name} does not exist.")
    else:
        for root, dirs, files in os.walk(dir_name):
            for file in files:
                if file == "all_metrics.json":
                    json_files.append(os.path.join(root, file))
json_files

Directory all_experiments/1Test/metrics does not exist.
Directory all_experiments/01Test/metrics does not exist.
Directory all_experiments/TMP_baby_stuck_noise/checkpoints/metrics does not exist.


['all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_034201/all_metrics.json',
 'all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.05_20250530_075401/all_metrics.json',
 'all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_035710/all_metrics.json',
 'all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_103931/all_metrics.json',
 'all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.05_20250530_060634/all_metrics.json',
 'all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_none_20250530_095709/all_metrics.json',
 'all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_014010/all_metrics.json',
 'all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_015506/all_metrics.json',
 'all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_122802/all_metrics.json',
 'all_ex

In [45]:
def flatten_config(config):
    flat = {}
    for k, v in config.items():
        if isinstance(v, list):
            flat[k] = str(v)  # convertir liste en string
        else:
            flat[k] = v
    return flat

In [46]:
all_data = []
for json_file in json_files:
    print(f"Trying to open {json_file}")
    with open(json_file, 'r') as f:
        data = json.load(f)

        flat_conf = flatten_config(data["config"])

        flat_results = {"train_acc": data["train_acc_history"], "val_acc": data["val_acc_history"], "noise_metrics": data["noise_metrics"]}
        merged = flat_conf | flat_results

        all_data.append(merged)

Trying to open all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_034201/all_metrics.json
Trying to open all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.05_20250530_075401/all_metrics.json
Trying to open all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_035710/all_metrics.json
Trying to open all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_103931/all_metrics.json
Trying to open all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.05_20250530_060634/all_metrics.json
Trying to open all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_none_20250530_095709/all_metrics.json
Trying to open all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_014010/all_metrics.json
Trying to open all_experiments/TMP_baby_stuck_noise/metrics/baby_cnn_weight_gaussian_0.01_20250530_015506/all_metrics.json
Trying to open all_experiments/T

In [47]:
df = pd.DataFrame(all_data)
df
pass

In [48]:
df["best_val_acc"] = df["val_acc"].apply(lambda x : max(x) if isinstance(x, list) else x)
df["last_val_acc"] = df["val_acc"].apply(lambda x : x[-1] if isinstance(x, list) else x)
df["best_train_acc"] = df["train_acc"].apply(lambda x : max(x) if isinstance(x, list) else x)
df["last_train_acc"] = df["train_acc"].apply(lambda x : x[-1] if isinstance(x, list) else x)
df["train_last_is_best"] = df["last_train_acc"] == df["best_train_acc"]
df["val_last_is_best"] = df["last_val_acc"] == df["best_val_acc"]
df

,data_dir,checkpoint_name,download_weights,classifier,model,batch_size,max_epochs,num_workers,gpu_id,learning_rate,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,permanent,noise_during_stuck_only,patience,subset,test_phase,train_acc,val_acc,noise_metrics,noise_layers,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
0,./data/cifar10,baby_cnn_1.0_sgd_weight_0.01_exponential.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.01,exponential,NaN,gaussian,sgd,0.0,0.0,0.0,False,True,5.0,1.0,False,"[0.5521233974358974, 0.5477764423076923, 0.552...","[0.5797275641025641, 0.5827323717948718, 0.590...","[{'epoch': 1, 'noise_type': 'NoiseType.weight'...",NaN,0.603866,0.602163,0.572416,0.571254,False,False
1,./data/cifar10,baby_cnn_1.0_sgd_weight_0.05_exponential.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.05,exponential,NaN,gaussian,sgd,0.0,0.0,0.0,False,True,5.0,1.0,False,"[0.5490184294871795, 0.5483173076923077, 0.553...","[0.5768229166666666, 0.5821314102564102, 0.588...","[{'epoch': 1, 'noise_type': 'NoiseType.weight'...",NaN,0.607672,0.606070,0.574760,0.574760,True,False
2,./data/cifar10,baby_cnn_1.0_sgd_weight_0.01_exponential.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.01,exponential,NaN,gaussian,sgd,0.0,0.0,0.0,False,False,5.0,1.0,False,"[0.548818108974359, 0.5487780448717949, 0.5500...","[0.5891426282051282, 0.5879407051282051, 0.591...","[{'epoch': 1, 'noise_type': 'NoiseType.weight'...",NaN,0.615585,0.614884,0.577464,0.577464,True,False
3,./data/cifar10,baby_cnn_1.0_sgd_weight_0.01_cosine.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.01,cosine,NaN,gaussian,sgd,0.9,0.0,0.0,False,False,5.0,1.0,False,"[0.549198717948718, 0.5455328525641026, 0.5450...","[0.5911458333333334, 0.5981570512820513, 0.603...","[{'epoch': 1, 'noise_type': 'NoiseType.weight'...",NaN,0.638722,0.638121,0.601402,0.601402,True,False
4,./data/cifar10,baby_cnn_1.0_sgd_weight_0.05_cosine.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.05,cosine,NaN,gaussian,sgd,0.0,0.0,0.0,False,False,5.0,1.0,False,"[0.46328125, 0.4652844551282051, 0.46500400641...","[0.5389623397435898, 0.5462740384615384, 0.547...","[{'epoch': 1, 'noise_type': 'NoiseType.weight'...",NaN,0.598958,0.597556,0.558694,0.558694,True,False
5,./data/cifar10,baby_cnn_1.0_sgd_baseline.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,none,0.01,constant,NaN,gaussian,sgd,0.9,0.0,0.0,False,False,5.0,1.0,False,"[0.5489182692307693, 0.5502203525641025, 0.547...","[0.5969551282051282, 0.5928485576923077, 0.611...",[],NaN,0.634415,0.632412,0.597175,0.597175,True,False
6,./data/cifar10,baby_cnn_1.0_sgd_weight_0.01_cosine.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.01,cosine,NaN,gaussian,sgd,0.0,0.0,0.0,False,True,5.0,1.0,False,"[0.5499399038461539, 0.5506410256410256, 0.552...","[0.5950520833333334, 0.6005608974358975, 0.599...","[{'epoch': 1, 'noise_type': 'NoiseType.weight'...",NaN,0.618890,0.617188,0.577224,0.574960,False,False
7,./data/cifar10,baby_cnn_1.0_sgd_weight_0.01_cosine.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.01,cosine,NaN,gaussian,sgd,0.0,0.0,0.0,False,False,5.0,1.0,False,"[0.5440304487179487, 0.5403846153846154, 0.544...","[0.5800280448717949, 0.5956530448717949, 0.600...","[{'epoch': 1, 'noise_type': 'NoiseType.weight'...",NaN,0.611478,0.610978,0.570212,0.568650,False,False
8,./data/cifar10,baby_cnn_1.0_sgd_weight_0.01_exponential.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.01,exponential,NaN,gaussian,sgd,0.9,0.0,0.0,False,True,5.0,1.0,False,"[0.5504407051282051, 0.5540464743589744, 0.550...","[0.5982572115384616, 0.5982572115384616, 0.601...","[{'epoch': 1, 'noise_type': 'NoiseType.weight'...",NaN,0.632913,0.631410,0.600361,0.600361,True,False
9,./data/cifar10,baby_cnn_1.0_sgd_weight_0.05_cosine.pth,0,baby_cnn,None,128,200,4,0,0.01,0.0001,weight,0.05,cosine,NaN,gaussian,sgd,0.0,0.0,0.0,False,True,5.0,1.0,False,"[0.56155849358974

In [49]:
constant_cols = []
for col in df.columns:
    try:
        if df[col].nunique() == 1:
            constant_cols.append(col)
    except TypeError:
        # ignorer colonnes non hashables
        pass

print("Colonnes constantes :", constant_cols)


Colonnes constantes : ['data_dir', 'download_weights', 'batch_size', 'num_workers', 'gpu_id', 'learning_rate', 'optimizer', 'beta1', 'beta2', 'permanent', 'patience', 'test_phase']


In [50]:
sub_df = df.copy()
sub_df.drop(columns=['data_dir', 'checkpoint_name', 'download_weights', 'model'], inplace=True)
sub_df.drop(columns=['batch_size', 'max_epochs', 'num_workers', 'gpu_id'], inplace=True)
sub_df.drop(columns=['learning_rate', 'test_phase',], inplace=True)
sub_df.drop(columns=['train_acc', 'val_acc', 'noise_metrics'], inplace=True)
sub_df
pass

In [51]:
cols = ['last_val_acc', 'classifier', 'noise_type', 'optimizer']

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_sorted = sub_df.sort_values(by=cols, ascending=False)
df_sorted

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,permanent,noise_during_stuck_only,patience,subset,noise_layers,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
15,resnet18,0.0100,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.928986,0.928486,0.990946,0.990204,False,False
14,resnet18,0.0100,gradient,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.887119,0.885216,0.915785,0.915785,True,False
13,resnet18,0.0100,gradient,0.03,constant,NaN,uniform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.802584,0.802584,0.805929,0.805929,True,True
16,resnet18,0.0100,gradient,0.05,constant,NaN,gaussian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.695012,0.695012,0.693670,0.693670,True,True
3,baby_cnn,0.0001,weight,0.01,cosine,NaN,gaussian,sgd,0.9,0.0,0.0,False,False,5.0,1.0,NaN,0.638722,0.638121,0.601402,0.601402,True,False
11,baby_cnn,0.0001,weight,0.01,cosine,NaN,gaussian,sgd,0.9,0.0,0.0,False,True,5.0,1.0,NaN,0.636919,0.634716,0.599439,0.598197,False,False
20,resnet18,0.0100,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,NaN,0.646334,0.633013,0.894832,0.889423,False,False
5,baby_cnn,0.0001,none,0.01,constant,NaN,gaussian,sgd,0.9,0.0,0.0,False,False,5.0,1.0,NaN,0.634415,0.632412,0.597175,0.597175,True,False
8,baby_cnn,0.0001,weight,0.01,exponential,NaN,gaussian,sgd,0.9,0.0,0.0,False,True,5.0,1.0,NaN,0.632913,0.631410,0.600361,0.600361,True,False
10,baby_cnn,0.0001,none,0.01,constant,NaN,gaussian,sgd,0.0,0.0,0.0,False,False,5.0,1.0,NaN,0.627003,0.625501,0.586298,0.583874,False,False


In [22]:
df_weight = df_sorted[(df_sorted['noise_type'] == 'weight')]
df_weight = df_weight.sort_values('noise_layer', ascending=False)

In [12]:
df_exp = df_sorted[(df_sorted['noise_schedule']=='exponential') | (df_sorted['noise_schedule'] == 'constant')]
df_exp = df_exp[df_exp['classifier'] == 'resnet18']
cols = ['noise_type','noise_layer', 'momentum']
df_exp.sort_values(cols, ascending=False)

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,permanent,noise_during_stuck_only,patience,subset,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best


In [14]:
df_no_momentum = df_exp[df_exp['momentum']==0.0]
df_no_momentum

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,subset,noise_layers,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
4,resnet18,0.001,weight,0.01,exponential,['bias'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.855569,0.853365,0.926783,0.926002,False,False
22,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.849860,0.848758,0.901362,0.901022,False,False
3,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.775441,0.775441,0.808694,0.808694,True,True
2,resnet18,0.001,weight,0.01,exponential,['bias'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.765224,0.762119,0.862430,0.862430,True,False
25,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.764123,0.746995,0.828325,0.828325,True,False
76,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.764123,0.746995,0.828325,0.828325,True,False
8,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.702123,0.696715,0.748898,0.748898,True,False
75,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.702123,0.696715,0.748898,0.748898,True,False
7,resnet18,0.001,none,0.01,constant,None,gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.852264,0.851462,0.926583,0.926362,False,False
13,resnet18,0.001,none,0.01,constant,None,gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.850361,0.850361,0.927364,0.927364,True,True


In [22]:
df_tinyCNN = df_sorted[df_sorted['classifier'] == 'tiny_cnn']
df_tinyCNN = df_tinyCNN[(df_tinyCNN['momentum'] == 0)|((df_tinyCNN['beta1'] == 0))]
df_tinyCNN = df_tinyCNN[df_tinyCNN['noise_type'] == 'gradient']
df_tinyCNN = df_tinyCNN[df_tinyCNN['noise_schedule'] == 'constant']
df_tinyCNN.sort_values('best_val_acc', ascending=False)




,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,subset,noise_layers,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
129,tiny_cnn,0.001,gradient,0.05,exponential,['layer4.'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.840345,0.839543,0.865705,0.865705,True,False
284,tiny_cnn,0.001,gradient,0.01,exponential,None,gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.839844,0.838842,0.870693,0.869371,False,False
239,tiny_cnn,0.001,gradient,0.01,exponential,['none'],gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.839143,0.838241,0.867969,0.867969,True,False
240,tiny_cnn,0.001,gradient,0.01,exponential,['none'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838842,0.838241,0.867428,0.866426,False,False
196,tiny_cnn,0.001,gradient,0.05,exponential,['none'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838442,0.837941,0.868770,0.868770,True,False
138,tiny_cnn,0.001,gradient,0.05,exponential,['bias'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838241,0.837941,0.865745,0.865745,True,False
153,tiny_cnn,0.001,gradient,0.05,exponential,['conv1'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.837941,0.835938,0.864944,0.864944,True,False
219,tiny_cnn,0.001,gradient,0.05,exponential,['none'],gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.836338,0.834435,0.867889,0.867889,True,False
142,tiny_cnn,0.001,gradient,0.01,exponential,['conv1'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.835837,0.834736,0.865365,0.863582,False,False
289,tiny_cnn,0.001,gradient,0.01,exponential,None,gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.835337,0.834635,0.869371,0.869371,True,False
